In [1]:
import json
import pandas as pd
import concurrent.futures

from airline_class import Flight

from pyensign.ensign import authenticate, publisher, subscriber
from FlightRadar24 import FlightRadar24API

# Configuration

In [2]:
ENSIGN_CREDS_PATH="ensign-creds.json"
FLIGHTRADAR_CREDS_PATH = "flightradar-creds.json"

def auth():    
    with open(FLIGHTRADAR_CREDS_PATH) as f:
        creds = json.load(f)
        user = creds["user"]
        password = creds["password"]
    return user, password
    
user, password = auth()
fr_api = FlightRadar24API(user=user, password=password)

In [3]:
callsigns = {}
training_list = []

# TODO: implement multithreading for faster execution of fr24 
# pool = concurrent.futures.ThreadPoolExecutor(max_workers=50)

In [4]:
def return_class_variables(Flight):
    return (Flight.__dict__)

In [5]:
def get_fr24_instance(airline_icao, icao24, latitude, longitude):
    # finding plane through tight bounds
    flight_bound = ",".join([str(latitude+0.25), str(latitude-0.25), str(longitude-0.25), str(longitude+0.25)])

    fr24_instance = fr_api.get_flights(airline=airline_icao, bounds=flight_bound)

# TODO: fix the bound tuner - process gets stuck inside here
    # limit = 0
    # while len(fr24_instance) != 1 or limit == 4:
    #     limit += 1
    #     if len(fr24_instance) < 1:
    #         flight_bound = ",".join([str(latitude+0.5), str(latitude-0.5), str(longitude-0.5), str(longitude+0.5)])
    #     if len(fr24_instance) > 1:
    #         flight_bound = ",".join([str(latitude+0.125), str(latitude-0.125), str(longitude-0.125), str(longitude+0.125)])
    #     fr24_instance = fr_api.get_flights(airline=airline_icao, bounds=flight_bound)
    
    if len(fr24_instance) == 0:
        return None

    origin_airport_icao = destination_airport_icao = scheduled_departure = scheduled_arrival = actual_departure = estimated_arrival = historical_flight_time = historical_delay = 0

    guard = [origin_airport_icao, destination_airport_icao, scheduled_departure, scheduled_arrival,
                        actual_departure, estimated_arrival, historical_flight_time, historical_delay]
    # get flight details
    for flight in fr24_instance:
        flight_details = fr_api.get_flight_details(flight)
        flight.set_flight_details(flight_details)

        # match correct instance
        if icao24.upper() != flight.icao_24bit:
            continue
        else:

            time_details = flight.time_details

            # airport
            origin_airport_icao = flight.origin_airport_icao
            destination_airport_icao = flight.destination_airport_icao

            # time
            scheduled_departure = time_details["scheduled"]["departure"]
            scheduled_arrival = time_details["scheduled"]["arrival"]
            actual_departure = time_details["real"]["departure"]
            estimated_arrival = time_details["estimated"]["arrival"]
            historical_flight_time = time_details["historical"]["flighttime"]
            historical_delay = time_details["historical"]["delay"]

        # guard
        for item in guard:
            if item == None:
                item = 0
    
    return {
        "origin_airport_icao" : origin_airport_icao,
        "destination_airport_icao" : destination_airport_icao,
        "scheduled_departure" : scheduled_departure,
        "scheduled_arrival" : scheduled_arrival,
        "actual_departure" : actual_departure,
        "estimated_arrival" : estimated_arrival,
        "historical_flight_time" : historical_flight_time,
        "historical_delay" : historical_delay
    }
    

In [6]:
# Removed SWA (Southwest Airlines) - fail to enter registration info

flight_convert = {
    "ASA": "Alaska Airlines", 
    "AAY": "Allegiant Air", 
    "AAL": "American Airlines", 
    "DAL": "Delta Air Lines",
    "FFT": "Frontier Airlines",	
    "HAL": "Hawaiian Airlines",	
    "JBU": "JetBlue",
    "NKS": "Spirit Airlines",
    "UAL": "United Airlines",
    "ENY": "Envoy Air",
    "RPA": "Republic Airways",
    "SKW": "SkyWest Airlines"
}


# Subscribe to Flight Updates

In [7]:
@authenticate(cred_path=ENSIGN_CREDS_PATH)
@subscriber("flight-positions")
async def get_updates(updates):
    async for flight in updates:
        await print_update(flight)
   
@publisher("Flights-json")
async def print_update(flight_event):
    flight = json.loads(flight_event.data)
    flight_instance = Flight(flight['icao24'])
    flight_instance.store_ensign_data(flight)
    
    airline_icao = flight_instance.airline_icao
    aircraft_icao = flight_instance.icao24

    # getting flightradar24 data
    # checking airline ICAO - only want major US carrier data
    if airline_icao in flight_convert:
        flight_instance.airline = flight_convert[airline_icao]
        print(flight_instance.callsign)
        fr24_instance = get_fr24_instance(airline_icao, aircraft_icao, flight_instance.latitude, flight_instance.longitude)
        
        if fr24_instance is not None:
            flight_instance.store_fr24_data(fr24_instance)
    
    flight_dict = return_class_variables(flight_instance)
    print(flight_dict)
    
    return flight_dict

In [8]:
await get_updates()


UAL1550 
{'icao24': 'aa56d8', 'callsign': 'UAL1550 ', 'airline_icao': 'UAL', 'origin_country': 'United States', 'time_position': 1697562620, 'last_contact': 1697562620, 'longitude': -97.4105, 'latitude': 32.2548, 'geo_altitude': 11765.28, 'on_ground': False, 'true_track': 133.6, 'velocity': 238.7, 'vertical_rate': 0.0, 'sensors': None, 'barometric_altitude': 11277.6, 'transponder_code': None, 'special_purpose_indicator': False, 'position_source': 0, 'category': 4, 'airline': 'United Airlines', 'origin_airport_icao': 'KPDX', 'destination_airport_icao': 'KIAH', 'scheduled_departure': 1697551200, 'scheduled_arrival': 1697566140, 'actual_departure': 1697551635, 'estimated_arrival': 1697564672, 'historical_flight_time': '12985', 'historical_delay': '-1173'}
{'icao24': 'a5f840', 'callsign': 'N484JC  ', 'airline_icao': 'N48', 'origin_country': 'United States', 'time_position': 1697562603, 'last_contact': 1697562603, 'longitude': -111.9008, 'latitude': 33.6307, 'geo_altitude': None, 'on_ground